# reviews.txt.gz conversion to .csv

### BeerAdvocate free-text reviews
As the BeerAdvocate text reviews are formatted inconveniently for the use of Pandas library directly,

the purpose of this ipynb notebook is to extract the .txt.gz file and convert it to a Pandas dateframe, 

which then later can be saved as in .csv format

In [1]:
#import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from lingua import LanguageDetectorBuilder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")


In [2]:
# Load the dataframe in which we have the reviews
df_reviews_ = pd.read_csv('reviews.csv.gz', compression='gzip')

In [3]:
# Load the language detector and sentiment analyzer
detector = LanguageDetectorBuilder.from_all_languages().build()
sia = SentimentIntensityAnalyzer()

# Load the brewery and user data
df_breweries = pd.read_csv('breweries.csv')
df_users = pd.read_csv('users.csv')

# Create dictionaries for the brewery and user data to map to the reviews
brewery_dict = dict(zip(df_breweries['name'], df_breweries['location'])) 
user_dict = dict(zip(df_users['user_name'], df_users['location']))

# Load the stopwords and add some additional ones to fit the theme - This is not all-encompassing, but it is hopefully good enough
stop_words = set(stopwords.words('english'))
additional_stop_words = ['beer', 'beers', 'tap', 'pour', 'pours', 'bottle', 'bottles',
                         'sample', 'cl', 'oz', 'ml', 'draft', 'glass', 'drink', 'drinks', 'shared', 'share', 'local', 'locals', 'aroma','smell','smells','smelling','smelled','taste','tastes','tasting','tasted','palate','mouthfeel','mouth','feel','feeling','feels',
                         'black', 'brown', 'red', 'yellow', 'white', 'orange', 'green', 'blue', 'purple', 'pink', 'color', 'colors','dark','light','lighter','darker','lightest','darkest','lightest','darkest','lightest','darkest','lightest','darkest','lightest','darkest',
                         'thanks', 'thx','thnx', 'flavor', 'flavour', 'flavors', 'flavours', 'note', 'notes', 'nose', 'noses', 'nosing', 'nosed', 'head', 'heads', 'heady', 'headier']
stop_words.update(additional_stop_words)

# Take the text from reviews, detect if it is english, tokenize it, and remove stopwords.
def clean_text(text):
    detection = detector.detect_language_of(text)

    if not detection:
        return None
    if detection.name == 'ENGLISH':
        words = text.split(' ')
        words = [word.lower() for word in words if word.isalpha()]
        words = [word for word in words if word not in stop_words]
        return words
    else:
        return None

# Check if the text is in English
def check_eng(text):
    if detector.detect_language_of(text).name == 'ENGLISH':
        return True
    else:
        return False

# Create a dataset with the average ratings for each beer
def create_avg_dataset(df_reviews_):
        
    # Average the reviews by beer_id
    df_reviews_['rating'] = df_reviews_['rating'].astype(float)

    # Sort out all of the beers with less than 30 reviews
    df_reviews_['count'] = df_reviews_.groupby('beer_id')['beer_id'].transform('count')
    df_reviews_less = df_reviews_[df_reviews_['count'] > 200]
    df_reviews_.drop('count', axis=1, inplace=True)

    # Average the ratings for each beer
    average_ratings = df_reviews_less.groupby(['beer_name', 'brewery_name'])['rating'].mean().reset_index()
    average_ratings.sort_values('rating', ascending=False, inplace=True)

    # Add location to average_ratings
    average_ratings['location'] = average_ratings['brewery_name'].map(brewery_dict)
    
    return average_ratings


def get_best_worst(average_ratings, country_from, country_to):

    # Get the best and worst beers from the country
    country_beers = average_ratings[average_ratings['location'].str.contains(country_from)]
    country_beers_best = country_beers.head(5)
    country_beers_worst = country_beers.tail(5)

    # Get the names of the best and worst beers
    best_beer_names = country_beers['beer_name'].values
    worst_beer_names = country_beers_worst['beer_name'].values

    # Get the text reviews for the best beer
    best_beer_reviews_df = df_reviews_[df_reviews_['beer_name'].str.contains('|'.join(list(country_beers_best['beer_name'])))]
    best_beer_reviews = best_beer_reviews_df['text']
    best_beer_reviews_df['user_location'] = best_beer_reviews_df['user_name'].map(user_dict)

    # Get the text reviews for the worst beer
    worst_beer_reviews_df = df_reviews_[df_reviews_['beer_name'].str.contains('|'.join(list(country_beers_worst['beer_name'])))]
    worst_beer_reviews = worst_beer_reviews_df['text']
    worst_beer_reviews_df['user_location'] = worst_beer_reviews_df['user_name'].map(user_dict)

    # Get the reviews from a specific country
    country_reviews_best = best_beer_reviews_df[best_beer_reviews_df['user_location'].str.contains(country_to, na=False)]
    country_reviews_worst = worst_beer_reviews_df[worst_beer_reviews_df['user_location'].str.contains(country_to, na=False)]

    # If there are no reviews from the country, we insert a row with the beer name, text, count = 0, and user_location
    if len(country_reviews_best) == 0:
        print("No 'Best' reviews from this country")
        country_reviews_best = pd.DataFrame({'beer_name': ['No Reviews'], 'text': ['No Reviews'], 'count': [0], 'user_location': [country_to]})
    if len(country_reviews_worst) == 0:
        print("No 'worst' reviews from this country")
        country_reviews_worst = pd.DataFrame({'beer_name': ['No Reviews'], 'text': ['No Reviews'], 'count': [0], 'user_location': [country_to]})

    return country_beers_best, country_beers_worst, \
        best_beer_names, worst_beer_names, country_reviews_best, \
            country_reviews_worst, best_beer_reviews, worst_beer_reviews, \
                best_beer_reviews_df, worst_beer_reviews_df


def get_word_counts(best_beer_names, worst_beer_names, country_reviews_best, country_reviews_worst):
    best_beer_counts = []
    worst_beer_counts = []
    for beer_names in zip(best_beer_names, worst_beer_names):
        # Extract the reviews for the beer
        reviews_best = country_reviews_best[country_reviews_best['beer_name'] == beer_names[0]]['text']
        reviews_worst = country_reviews_worst[country_reviews_worst['beer_name'] == beer_names[1]]['text']

        # Clean the text and drop the NaN values such that we are left with only english words
        cleaned_text_best = reviews_best.apply(clean_text)
        cleaned_text_best.dropna(inplace=True)
        cleaned_text_worst = reviews_worst.apply(clean_text)
        cleaned_text_worst.dropna(inplace=True)
        #print(cleaned_text_worst)

        # Count the words in the reviews
        all_words_best = []
        all_words_worst = []
        for words in cleaned_text_best:
            all_words_best += words
        count = Counter(all_words_best)
        best_beer_counts.append(count)

        # Count the words in the reviews
        for words in cleaned_text_worst:
            all_words_worst += words
        count = Counter(all_words_worst)
        worst_beer_counts.append(count)
    
    return best_beer_counts, worst_beer_counts


def make_final_df(country_beers_best, country_beers_worst, best_beer_counts, worst_beer_counts, best_beer_reviews_df, worst_beer_reviews_df):
    best_beers_final_df = pd.DataFrame()

    # For each beer, we get the 10 most common words and their counts
    for i, beer_name in enumerate(list(country_beers_best['beer_name'])):
        count = best_beer_counts[i]
        count = dict(sorted(count.items(), key=lambda item: item[1], reverse=True)[:10])
        count = pd.DataFrame(count.items(), columns=['word', 'count'])
        count['beer_name'] = f"#{i+1} " + beer_name
        count['best_worst'] = 'Best'
        
        # If there are no reviews, then we insert a row such that it still pops up in the tree map, but with a single entry 'No Reviews'
        if len(best_beer_counts[i]) == 0:
            count = pd.DataFrame({'word': ['No Reviews'], 'count': [1], 'beer_name': [f"#{i+1} " + beer_name], 'best_worst': ['Best'], 'Positivity percentage': [0.5], 'Negativity percentage': [0.5]})
        else:
            positive_scores = []
            negative_scores = []
            
            # For each word, we calculate the positivity and negativity percentage of that word in context of all reviews
            # This gives us an idea if a word is generally used in a positive or negative context - There is some room for improvmenent here.
            for word in count['word']:
                pos = 0
                neg = 0
                #neu = 0
                for review in best_beer_reviews_df[best_beer_reviews_df['beer_name'] == beer_name]['text']:
                    if word in review and check_eng(review):
                        sentiment = sia.polarity_scores(review)
                        pos += sentiment['pos']
                        neg += sentiment['neg']
                        # We decided to not use neutral sentiment, as it is not as informative/fun as the other two - The sentiment for this is often too high
                        #neu += sentiment['neu']
                    
                    # Normalize these to add up to 1
                    total_score = pos + neg #+ neu
                    if total_score != 0:
                        pos /= total_score
                        neg /= total_score
                        #neu /= total_score


                positive_scores.append(pos)
                negative_scores.append(neg)
                #neutral_scores.append(neu)
            count['Positivity percentage'] = positive_scores
            count['Negativity percentage'] = negative_scores
            #count['neutral'] = neutral_scores
            #print(count)
        best_beers_final_df = pd.concat([best_beers_final_df, count], ignore_index=True)
    # END BEST
    # The exact same thing is done for the worst beers
    worst_beers_final_df = pd.DataFrame()

    for i, beer_name in enumerate(list(country_beers_worst['beer_name'])):
        count = worst_beer_counts[i]
        count = dict(sorted(count.items(), key=lambda item: item[1], reverse=True)[:10])
        count = pd.DataFrame(count.items(), columns=['word', 'count'])
        count['beer_name'] = f"#{i+1} " + beer_name
        count['best_worst'] = 'Worst'

        if len(worst_beer_counts[i]) == 0:
            count = pd.DataFrame({'word': ['No Reviews'], 'count': [1], 'beer_name': [f"#{i+1} " + beer_name], 'best_worst': ['Worst'], 'Positivity percentage': [0.5], 'Negativity percentage': [0.5]})
        else:
            positive_scores = []
            negative_scores = []
            for word in count['word']:
                pos = 0
                neg = 0
                #neu = 0
                for review in worst_beer_reviews_df[worst_beer_reviews_df['beer_name'] == beer_name]['text']:
                    if word in review and check_eng(review):
                        sentiment = sia.polarity_scores(review)
                        pos += sentiment['pos']
                        neg += sentiment['neg']
                        #neu += sentiment['neu']
                    
                    # Normalize these to add up to 1
                    total_score = pos + neg# + neu
                    if total_score != 0:
                        pos /= total_score
                        neg /= total_score
                        #neu /= total_score

                positive_scores.append(pos)
                negative_scores.append(neg)
                #neutral_scores.append(neu)
            count['Positivity percentage'] = positive_scores
            count['Negativity percentage'] = negative_scores
        #count['neutral'] = neutral_scores
 
        worst_beers_final_df = pd.concat([worst_beers_final_df, count], ignore_index=True)

    #print(worst_beers_final_df.to_string())
    #print(best_beers_final_df.to_string())
    
    final_df = pd.concat([best_beers_final_df, worst_beers_final_df], ignore_index=True)
        
    return final_df


def create_df_ez(average_ratings, country_from, country_to):
    # Use all of the beforementioned functions to create a dataframe we want easily
    country_beers_best, country_beers_worst, \
        best_beer_names, worst_beer_names, country_reviews_best, \
            country_reviews_worst, _, _, \
            best_beer_reviews_df, worst_beer_reviews_df = get_best_worst(average_ratings, country_from, country_to)

    best_beer_counts, worst_beer_counts = get_word_counts(best_beer_names, worst_beer_names, country_reviews_best, country_reviews_worst)

    final_df = make_final_df(country_beers_best, country_beers_worst, best_beer_counts, worst_beer_counts, best_beer_reviews_df, worst_beer_reviews_df)
    
    # Add data for the tree map plot to make the hierarchy work
    final_df['All countries'] = 'All countries'
    final_df['Review from'] = f"Reviews from {country_to}"
    final_df['Beers from'] = f"Beer from {country_from}"
    
    return final_df

In [4]:
# Make the dataframe as needed - only has to be done once
average_ratings = create_avg_dataset(df_reviews_)

In [9]:
# The deepdrive countries initially chosen
deepdive_countries = ['United States', 'Denmark', 'Spain', 'Belgium', 'Germany', 'Italy',
                      'France', 'England', 'Canada', 'Australia']

country_combs = []

# Create all the combinations of countries we want to compare
for country in deepdive_countries:
    for country2 in deepdive_countries:
            country_combs.append((country, country2))

In [10]:
# Create the hierarchical dataframe with just the initial countries
# This takes quite long as we need to process 10^2 combinations
"""
huge_df = pd.DataFrame()

for country_from, country_to in country_combs:
    print("Now starting with ", country_from, country_to)
    final_df = create_df_ez(average_ratings, country_from, country_to)
    huge_df = pd.concat([huge_df, final_df], ignore_index=True)
    print("Done with", country_from, country_to, " Only", len(country_combs) - country_combs.index((country_from, country_to)), "left \n")
"""

'\nhuge_df = pd.DataFrame()\n\nfor country_from, country_to in country_combs:\n    print("Now starting with ", country_from, country_to)\n    final_df = create_df_ez(average_ratings, country_from, country_to)\n    huge_df = pd.concat([huge_df, final_df], ignore_index=True)\n    print("Done with", country_from, country_to, " Only", len(country_combs) - country_combs.index((country_from, country_to)), "left \n")\n'

In [11]:
# Some extra countries added to the mix
extra_countries = ['Sweden', 'Norway']

all_countries = deepdive_countries + extra_countries

all_combs = []

# Create all the combinations of countries we want to compare
for country in all_countries:
    for country2 in all_countries:
        all_combs.append((country, country2))

# Filter out only such that the combs contain those of the extra countries
all_combs = [comb for comb in all_combs if comb[0] in extra_countries or comb[1] in extra_countries]

In [12]:
# Create the hierarchical dataframe with just the extra added countries
# this takes quite long as well as we need to process the extra combinations
"""
huge_df_3 = pd.DataFrame()
for country_from, country_to in all_combs:
    print("Now starting with ", country_from, country_to)
    final_df = create_df_ez(average_ratings, country_from, country_to)
    huge_df_3 = pd.concat([huge_df_3, final_df], ignore_index=True)
    print("Done with", country_from, country_to, " Only", len(all_combs) - all_combs.index((country_from, country_to)), "left \n")

huge_df_3.to_csv('huge_df_3.csv', index=False)
"""

Now starting with  United States Sweden
Done with United States Sweden  Only 44 left 

Now starting with  United States Norway
Done with United States Norway  Only 43 left 

Now starting with  Denmark Sweden
Done with Denmark Sweden  Only 42 left 

Now starting with  Denmark Norway
Done with Denmark Norway  Only 41 left 

Now starting with  Spain Sweden
Done with Spain Sweden  Only 40 left 

Now starting with  Spain Norway
Done with Spain Norway  Only 39 left 

Now starting with  Belgium Sweden
Done with Belgium Sweden  Only 38 left 

Now starting with  Belgium Norway
Done with Belgium Norway  Only 37 left 

Now starting with  Germany Sweden
Done with Germany Sweden  Only 36 left 

Now starting with  Germany Norway
Done with Germany Norway  Only 35 left 

Now starting with  Italy Sweden
Done with Italy Sweden  Only 34 left 

Now starting with  Italy Norway
Done with Italy Norway  Only 33 left 

Now starting with  France Sweden
Done with France Sweden  Only 32 left 

Now starting with  

In [13]:
# Save the final dataframe to a csv
huge_df = pd.read_csv('huge_df.csv')
#huge_df_2 = pd.read_csv('huge_df_2.csv')
huge_df_3 = pd.read_csv('huge_df_3.csv')

huge_df = pd.concat([huge_df, huge_df_3], ignore_index=True)
#huge_df = pd.concat([huge_df, huge_df_3], ignore_index=True)

# Drop the rows which has Vietnam or Czech Republic or Spain as the country to and from
    # Initiall we had these two countries but decided they were too insignificant in terms of reviews
huge_df = huge_df[~huge_df['Review from'].str.contains('Vietnam|Czech Republic|Spain', na=False)]
huge_df = huge_df[~huge_df['Beers from'].str.contains('Vietnam|Czech Republic|Spain', na=False)]

In [14]:
# Change the column name for Positivity percentage to "Positivity %"
huge_df.rename(columns={'Positivity percentage': 'Positivity %'}, inplace=True)

In [15]:
# Create the treemap using Plotly - It has the following architecture
#testdf
# Reviews from ->
# Beers From ->
# [Best -> Beer names 
#           [Beer name #1 -> Words
#           [Beer name #2 -> Words ...

# [Worst -> Beer names
#           [Beer name #1 -> Words
#           [Beer name #2 -> Words ...

fig = px.treemap(huge_df, path=['All countries','Beers from','Review from', 'best_worst', 'beer_name', 'word'], 
                 values='count', 
                 color='Positivity %', color_continuous_scale='Oranges', 
                 title='', 
                 hover_data={'Positivity %':True, 'Negativity percentage':True}, color_continuous_midpoint=0.5,
                 maxdepth=2,
                 hover_name='Positivity %',
                 )#,
                 #hovertemplate='<b>%{count}<br>')

fig.update_traces(root_color="Black", 
                  hovertemplate='Word occurence in reviews: %{value:.0f}<br>Percentage of times word is positive: %{color}<br><extra>%{label}</extra>')
fig.data[0]['textfont']['size'] = 15

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))

# Save fig
fig.write_html('Tree_plot.html')

fig.show()